## Notebook for visualizing results of script `rf_3dmigrate.py`

In [ ]:
import os
import os.path

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from obspy import read_inventory, read_events, UTCDateTime as UTC
from obspy.clients.fdsn import Client
from rf import read_rf, RFStream
from rf import get_profile_boxes, iter_event_data, IterMultipleComponents
from rf.imaging import plot_profile_map
from rf.profile import profile
from tqdm import tqdm
# from eqcorrscan.utils.stacking import linstack, PWS_stack, align_traces
import rf
import scipy
from scipy.spatial import cKDTree
from scipy.interpolate import interp1d
from pyevtk.hl import gridToVTK

In [ ]:
input_folder = '/g/data/ha3/am7399/shared/OA_RF_analysis/3dmigrate'
output_folder = '/g/data/ha3/am7399/shared/OA_RF_analysis/3dmigrate'

In [ ]:
# --dimensions 800 400 100 --num-cells 80 40 500
cz = np.loadtxt(os.path.join(input_folder, 'cz.txt'))
vol = np.loadtxt(os.path.join(input_folder, 'vol.txt'))
volHits = np.loadtxt(os.path.join(input_folder, 'volHits.txt'))
gxaa = np.loadtxt(os.path.join(input_folder, 'gxaa.txt'))
gyaa = np.loadtxt(os.path.join(input_folder, 'gyaa.txt'))
gzaa = np.loadtxt(os.path.join(input_folder, 'gzaa.txt'))
glon = np.loadtxt(os.path.join(input_folder, 'glon.txt'))
glat = np.loadtxt(os.path.join(input_folder, 'glat.txt'))

nx = 80
ny = 40
nz = 500

In [ ]:
cz = cz.reshape((nx-1,ny-1,nz-1))
vol = vol.reshape((nx-1,ny-1,nz-1))
volHits = volHits.reshape((nx-1,ny-1,nz-1))
gxaa = gxaa.reshape((nx,ny,nz))
gyaa = gyaa.reshape((nx,ny,nz))
gzaa = gzaa.reshape((nx,ny,nz))

glon = glon.reshape((nx,ny,nz))
glat = glat.reshape((nx,ny,nz))

In [ ]:
plt.figure(figsize=(20,10))

pgx = (gxaa[:-1,0,0] + gxaa[1:,0,0])/2
pgz = (gzaa[0,0,:-1] + gzaa[0,0,1:])/2

pgx,pgz = np.meshgrid(pgx,pgz,indexing='ij')

print(pgx.shape)
#meanVol = np.mean(vol*cz**1.5, axis=1)
meanVol = np.mean(vol[:,0:10,:]*cz[:,0:10,:]**2, axis=1)
print(meanVol.shape)

#meanVol /= np.sqrt(np.mean(np.power(meanVol,2)))
plt.contourf(pgx, pgz, np.sum(volHits[:,10:15,:], axis=1), 50, cmap='jet')

norm = mpl.colors.Normalize(vmin=np.min(meanVol), vmax=-np.min(meanVol))

pgxz = scipy.ndimage.zoom(pgx, 3)
pgzz = scipy.ndimage.zoom(pgz, 3)
meanVolz = scipy.ndimage.zoom(meanVol, 3)

plt.contourf(pgxz, pgzz, meanVolz, 500, norm=norm, cmap='seismic')
#plt.contourf(pgx, pgz, meanVol, 500, norm=norm, cmap='seismic')


#meanCz = np.mean(cz, axis=1)
meanCz = np.mean(cz[:,30:35,:], axis=1)

norm = mpl.colors.Normalize(vmin=np.min(meanCz), vmax=np.max(meanCz))

pgxz = scipy.ndimage.zoom(pgx, 3)
pgzz = scipy.ndimage.zoom(pgz, 3)
meanCzz = scipy.ndimage.zoom(meanCz, 3)

#plt.contourf(pgxz, pgzz, meanCzz, 500, norm=norm, cmap='seismic')
#plt.contourf(pgx, pgz, meanCzz, 500, norm=norm, cmap='seismic')



plt.colorbar()

plt.axes().set_xlabel('Distance along azimuth [km]')
plt.axes().set_ylabel('Depth [km]')
plt.tick_params(axis='both', labelsize=15)

plt.axes().set_ylim([0,50])
plt.axes().invert_yaxis()
plt.axes().set_aspect(7)

plt.show()


In [ ]:
gxaac = (gxaa[:-1, :-1, :-1] + gxaa[1:, 1:, 1:]) / 2.
gyaac = (gyaa[:-1, :-1, :-1] + gyaa[1:, 1:, 1:]) / 2.
gzaac = (gzaa[:-1, :-1, :-1] + gzaa[1:, 1:, 1:]) / 2.

xyzs = np.array([gxaac[:,:,0].flatten(), gyaac[:,:,0].flatten(), gzaac[:,:,0].flatten()]).T
tree = cKDTree(xyzs)

In [ ]:
rIO = interp1d(gzaac[0,0,:], np.linspace(30,100, nz-1))

smoothVolLi = np.zeros(vol.shape)
smoothVolSq = np.zeros(vol.shape)
smoothVolCu = np.zeros(vol.shape)
for ix in np.arange(nx-1):
    for iy in np.arange(ny-1):
        for iz in np.arange(nz-1):
            ids = np.array(tree.query_ball_point([gxaac[ix,iy,iz], gyaac[ix,iy,iz], gzaac[ix,iy,iz]], 
                                                 r=rIO(gzaac[ix,iy,iz])))
            #print len(ids)
            lix = np.floor_divide(ids, ny-1)
            liy = np.mod(ids, ny-1)
            #print lix
            #print liy
            if(len(ids)):
                #ids += lix*(ny-1)*(nz-1) + liy*(nz-1)
                smoothVolLi[ix,iy,iz] = np.mean(vol[lix, liy, iz]*cz[lix, liy, iz])
                smoothVolSq[ix,iy,iz] = np.mean(vol[lix, liy, iz]*cz[lix, liy, iz]**2)
                smoothVolCu[ix,iy,iz] = np.mean(vol[lix, liy, iz]*cz[lix, liy, iz]**3)

In [ ]:
rIO = interp1d(gzaac[0,0,:], np.linspace(30,100, nz-1))
plt.plot(gzaac[0,0,:], rIO(gzaac[0,0,:]))
plt.show()

In [ ]:
plt.figure(figsize=(20,10))

pgx = (gxaa[:-1,0,0] + gxaa[1:,0,0])/2
pgz = (gzaa[0,0,:-1] + gzaa[0,0,1:])/2

pgx,pgz = np.meshgrid(pgx,pgz,indexing='ij')

meanVol = np.mean(smoothVolSq[:,25:35,:], axis=1)

norm = mpl.colors.Normalize(vmin=np.min(meanVol), vmax=-np.min(meanVol))

plt.contourf(pgx, pgz, meanVol, 500, norm=norm, cmap='seismic')
#plt.colorbar()

plt.axes().set_xlabel('Distance along azimuth [km]')
plt.axes().set_ylabel('Depth [km]')
plt.tick_params(axis='both', labelsize=15)
plt.axes().set_ylim([0,50])
plt.axes().invert_yaxis()
plt.axes().set_aspect(5)
plt.tight_layout()
#plt.savefig('/tmp/linweighting.png', dpi=300)
plt.savefig(os.path.join(output_folder, 'hCC.png'), dpi=300)


In [ ]:
plt.figure(figsize=(20,10))

pgy = (gyaa[0,:-1,0] + gyaa[0,1:,0])/2
pgz = (gzaa[0,0,:-1] + gzaa[0,0,1:])/2

pgy,pgz = np.meshgrid(pgy,pgz,indexing='ij')

meanVol = np.mean(smoothVolSq[25:35,:,:], axis=0)

norm = mpl.colors.Normalize(vmin=np.min(meanVol), vmax=-np.min(meanVol))

plt.contourf(pgy, pgz, meanVol, 500, norm=norm, cmap='seismic')
#plt.colorbar()

plt.axes().set_xlabel('Distance along crossline direction [km]')
plt.axes().set_ylabel('Depth [km]')
plt.tick_params(axis='both', labelsize=15)

plt.axes().set_ylim([0,50])
plt.axes().invert_yaxis()
plt.axes().set_aspect(5)
plt.tight_layout()

plt.savefig(os.path.join(output_folder, 'vCC.png'), dpi=300)


In [ ]:
plt.figure(figsize=(20,10))

pgx = (gxaa[:-1,0,0] + gxaa[1:,0,0])/2
pgy = (gyaa[0,:-1,0] + gyaa[0,1:,0])/2

pgx,pgy = np.meshgrid(pgx,pgy,indexing='ij')

meanVol = np.mean(smoothVolSq[:,:,150:151], axis=2)

norm = mpl.colors.Normalize(vmin=np.min(meanVol), vmax=-np.min(meanVol))

plt.contourf(pgx, pgy, meanVol, 500, norm=norm, cmap='seismic')
#plt.colorbar()

plt.axes().set_xlabel('Distance along inline direction [km]')
plt.axes().set_ylabel('Distance along crossline direction [km]')
plt.tick_params(axis='both', labelsize=15)
plt.axes().invert_yaxis()

plt.tight_layout()

plt.savefig(os.path.join(output_folder, 'd.png'), dpi=300)


In [ ]:
gridToVTK("./structuredg", 
          glon, glat, np.array(gzaa), 
          cellData = {"Amplitude":np.array(smoothVolSq), 
                      "VolHits":np.array(volHits)})

In [ ]:
print(gzaa)